# Scrapping vacina COVID-19

Depois de algumas (várias) tentativas consegui desenvolver um código não muito complexo e com muita aplicabilidade para gerar os dados de vacina de COVID-19 para os estados Brasileiros, esse código faz parte do painel COVID-19 que eu estou montando em paralelo com auxilio do streamlit e que está ficando bem legal (com previsão de estar na web em Maio), o que faz com que o output desse código (o agrupamento) funcione para alguns e não para outros. 

O intuito aqui é compartilhar o código bruto e permitir que as pessoas tenham acesso aos dados da forma que quiserem, simbora.

Os dados são os dados do OpenDatasus de vacinação COVID-19 onde cada linha é uma pessoa e as informações referentes a ela
O link é este: https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8

Fiz um loop para puxar cada estado de uma vez pois as tentativas de puxar "Dados Completos" foram falhas, mas é isso, se tiverem dicas de melhorar eu quero super!


Pacotes que eu uso aqui:
* Pandas (para colocar em dataframe as informações extraídas)
* Io (auxilia em extrair junto com o requests)
* Requests (puxar os dados da URL)
* Numpy (uso no agrupamento)




Nesse exemplo eu trouxe as variáveis de maior relevância e abaixo descrevo o que são:

* paciente_endereco_nmMunicipio: nome do municipio do vacinado

* paciente_enumSexoBiologico: sexo biologio, se feminino ou masculino

* vacina_categoria_nome: categoria do vacinado (se por idade, trabalhador de saúde, limpeza)

* vacina_descricao_dose: dose da vacina, se primeira ou segunda dose

* paciente_racaCor_valor: raça/cor do vacinado

* UF: estado (eu criei essa variável)

* vacina_dataAplicacao: data de aplicação da vacina

In [ ]:
import pandas as pd 
import io
import requests
import numpy as np


results_caract = [] # guardar os resultados
results_series = [] # guardar os resultados

seq_url = ['AC','AL','AM','AP','BA','CE','DF','ES','GO','MA','MG','MS','MT','PA','PB','PE','PI','PR','RJ','RN','RO','RR','RS','SC','SE','SP', 'TO'] # ufs brasileiras

for i in seq_url:
    i = str(i)
    url = "https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/PNI/vacina/uf/2021-04-04/uf%3D"+ i + "/part-00000-072d96f6-f1d2-4b7f-a602-a64093bf2cf5.c000.csv" # inspecionei o codigo e vi que estao nesse link
    print('Estado sendo extraído: '+ i) # printo o estado que esta sendo extraido
    s=requests.get(url).content # requiro as informações
    c=pd.read_csv(io.StringIO(s.decode('utf-8')), sep = ';') # coloco as informaçõe em data frame e arrumo o encoding
    c['UF'] = i # trago a informação da UF
    c['indicador'] = 1 # crio indicador para auxiliar contagem

    caract = c.groupby(['paciente_endereco_nmMunicipio', "paciente_enumSexoBiologico", "vacina_categoria_nome","vacina_descricao_dose", 'paciente_racaCor_valor', 'UF' ]).agg({"paciente_idade": [np.mean, 'count']}).reset_index() # agrupo o output como EU quero, aqui pode nao valer para alguns de voces
    time = c.groupby(['paciente_endereco_nmMunicipio','vacina_dataAplicacao', 'UF']).agg({"indicador": [ np.sum]}).reset_index()  # agrupo o output como EU quero, aqui pode nao valer para alguns de voces


    results_caract.append(caract) # append de todos os estados no  meu output
    results_series.append(time) # append de todos os estados no meu ouput

In [ ]:
results_caract = pd.concat(results_caract)  # concatenando
results.to_csv('caract_vac.csv', index = False) # salvando a base


In [ ]:
results_caract = pd.concat(results_series) # concatenando
results_series.to_csv('time_vac.csv', index = False) # salvando a base